In [247]:
from sklearn.decomposition import randomized_svd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [248]:
# user = pd.read_csv('../Data/whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
# product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
# video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
# user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
# user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # shape(31176,5834) 유저 x 상품 아이템
# user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # shape (31177, 3250) 유저 x 영상 아이템
#null_del_user_place = pd.read_csv('../Data/whyout_data/null_del_user_place.csv')
# null_del_user_product = pd.read_csv('../Data/whyout_data/null_del_user_product.csv')
# null_del_user_video = pd.read_csv('../Data/whyout_data/null_del_user_video.csv')
index_null_del_user_place = pd.read_csv('../Data/whyout_data/index_null_del_user_place.csv', index_col=0)
# index_null_del_user_product = pd.read_csv('../Data/whyout_data/index_null_del_user_product.csv', index_col=0)
# index_null_del_user_video = pd.read_csv('../Data/whyout_data/index_null_del_user_video.csv', index_col=0)
# normal_index_null_del_user_place = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_place.csv', index_col=0)
# normal_index_null_del_user_product = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_product.csv', index_col=0)
# normal_index_null_del_user_video = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_video.csv', index_col=0)

In [249]:
# 데이터프레임을 numpy matrix로 만듦
user_product_matrix = index_null_del_user_place.values

# 사용자의 평균 평점
user_ratings_mean = np.mean(user_product_matrix, axis=1)

# 사용자-상품에 대해 사용자 평균 평점을 뺀 것
matrix_user_mean = user_product_matrix - user_ratings_mean.reshape(-1,1)

In [250]:
pd.DataFrame(matrix_user_mean, columns = index_null_del_user_place.columns).head()

,0,1,2,3,4,5,6,7,8,9,...,4687,4688,4689,4690,4691,4692,4693,4694,4695,4696
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,0.999787,-0.000213,-0.000213,...,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213,-0.000213
2,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,1.993826,-0.006174,-0.006174,-0.006174,-0.006174,...,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174,-0.006174
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,2.984671,-0.015329,-0.015329,0.984671,-0.015329,...,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329,-0.015329


In [251]:
# U 행렬, sigma 행렬, V 전치 행렬을 반환.
# U, sigma, Vt = svds(matrix, k = 20)
U, sigma, Vt = randomized_svd(user_product_matrix, n_components=20)

# 특이값 벡터를 대각 행렬 형태로 변환해줌
sigma = np.diag(sigma)

In [252]:
# U, Sigma, Vt의 행렬곱을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 + 사용자 평균 rating을 적용한다. 
svd_user_predicted_ratings = np.matmul(np.matmul(U,sigma), Vt) + user_ratings_mean.reshape(-1,1)

In [253]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = index_null_del_user_place.columns)

In [280]:
def recommend_movies(df_svd_preds, user_id, item_df, ratings_df, num_recommendations):
    """
    :param df_svd_preds: SVD 예측 평점 데이터프레임
    :param user_id: 사용자 ID
    :param df: 아이템 데이터프레임
    :param ratings_df: 사용자 평점 데이터프레임
    :param num_recommendations: 추천할 아이템 수
    :return: 사용자가 이미 평가한 아이템 인덱스와 추천 아이템 데이터프레임
    """
    # 원본 평점 데이터에서 user_id에 해당하는 행을 Series 형태로 가져오기
    user_data = ratings_df.loc[user_id]

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]

    # SVD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    user_row_number = user_id - 1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = item_df.loc[~item_df.index.isin(user_history_indices)]

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    predictions_df.set_index('index', inplace=True)

    # recommendations의 인덱스와 predictions_df의 인덱스를 기준으로 병합
    recommendations = recommendations.merge(predictions_df, left_index=True, right_index=True)

    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]
    print(f"user {user_id}에게 추천해줄 {num_recommendations}개 아이템 id : {recommendations.index.values}")

    return user_history_indices, recommendations

# 함수 사용 예시
already_rated, predictions = recommend_movies(df_svd_preds, 10, place, index_null_del_user_place, 10)

user 10에게 추천해줄 10개 아이템 id : [3348 1384 3457 3671 3283 2891 3427 1752 2265 3892]


In [281]:
predictions

,idx,지역(위치정보),공간유형,주변환경,계절,테마,클릭 수,좋아요 수,북마크 수,댓글 수,Predictions
3348,3350,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,0,1,0,0,0,0]",NaN,"[1,1,1,1,1,0,0,0,0,0,0,0,0,1,0]",0.326868,0.270270,0.326633,0.222222,0.038208
1384,1385,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,1,0,1,0,1,1,0,1,0]",0.284178,0.324324,0.320352,0.000000,0.038104
3457,3460,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,1,0,0,0,0,0,0,1,0]",0.254262,0.193694,0.236181,0.111111,0.029607
3671,3674,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,0,0,0,0,0,0]",NaN,"[1,1,1,1,0,0,1,0,1,0,0,1,0,0,1]",0.258442,0.225225,0.316583,0.000000,0.028677
3283,3284,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,0,1,0,0,1,0,0,0,0,1,0,1,0]",0.171512,0.090090,0.173367,0.000000,0.027563
2891,2892,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,1,0,0,0,0,0,0]",NaN,"[1,0,1,1,0,0,1,0,0,1,0,0,0,1,0]",0.215423,0.162162,0.164573,0.111111,0.024776
3427,3429,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,0,0,0,0,0,1,0,1,0]",0.197060,0.202703,0.228643,0.000000,0.022956
1752,1753,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,1,0,0,0,0,0,0]","[0,1,0,0,0,0,0,0]","[1,1,1,0]","[1,1,1,1,1,1,0,0,0,1,0,0,0,0,0]",0.213685,0.148649,0.157035,0.000000,0.022452
2265,2266,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,1,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,0,0,1,1,0,1,1,1,0]",0.252759,0.202703,0.251256,0.000000,0.017593
3892,3900,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,1,0,0,0,0,0,0]","[0,1,0,1,0,0,0,0]",NaN,"[1,1,1,1,1,0,1,0,0,0,1,0,0,0,1]",0.234068,0.153153,0.208543,0.111111,0.016318
